In [1]:
!pip install Bio

     |████████████████████████████████| 269 kB 7.1 MB/s 
     |████████████████████████████████| 2.3 MB 49.2 MB/s 


In [2]:
import math

In [3]:
prefixMap = {}

In [4]:
suffixList = []


In [5]:
encode = {}

encode["A"] = 1
encode["C"] = 2
encode["T"] = 3
encode["G"] = 4

In [6]:
prime = 1073741789

In [7]:
def rabinKarpPrefix(seq,readID):
    iterations = 17
    start = 0
    length = 85

    hash = 0

    for i in range(85):
        x = encode[seq[84-i]]
        hash += (x*math.pow(5,i)) % prime

    k = (hash,length)
    prefixMap.setdefault(k, [])
    prefixMap[k].append(readID)


    for i in range(16):

        y = encode[seq[85+i]]
        hash = (5*hash + y) % prime
        key = (hash,length+i)
        prefixMap.setdefault(key, [])
        prefixMap[key].append(readID)

    
        
        
        

In [8]:
def rabinKarpSuffix(seq,readID):

   

    
    length = 85

    hash = 0

    for i in range(85):
        x = encode[seq[100-i]]
        hash += (x*math.pow(5,i)) % prime

    suffixList.append([hash,length,readID])


    for i in range(16):
        y = encode[seq[15-i]]
        hash += (5*y) % prime
        suffixList.append([hash,length+i+1,readID])

In [9]:
from Bio import SeqIO
readID =0

fasta_sequences = SeqIO.parse(open('/content/drive/MyDrive/DeNovo/reads.fasta'),'fasta')
for fasta in fasta_sequences:
  name, sequence = fasta.id, str(fasta.seq)
  reverseComplement = str(fasta.seq.reverse_complement())
  
  # Code Snippet credit for parsing and reverse complement:
  # https://biopython.org/wiki/SeqIO
  readIdOfRead = 2*readID
  readIdOfComplement = 2*readID + 1

  rabinKarpPrefix(sequence,readIdOfRead)
  rabinKarpSuffix(sequence,readIdOfRead)

  rabinKarpPrefix(reverseComplement,readIdOfComplement)
  rabinKarpSuffix(reverseComplement,readIdOfComplement)
  
  readID += 1
  

In [ ]:
len(prefixMap)

19220356

In [ ]:
readID

574846

In [10]:
r = 2*readID

In [11]:
graph = [[]]*r

In [ ]:
suffixList[0]

[37751230606.0, 85, 0]

In [12]:
sortedSuffixList = sorted(suffixList, key=lambda x: x[1], reverse = True)

In [13]:
for fp,lenOfSuffix,srcReadID in sortedSuffixList:
  if((fp,lenOfSuffix) in prefixMap):
    Lp = prefixMap[(fp,lenOfSuffix)]
    l = []
    for destReadID in Lp:
      l.append([destReadID,lenOfSuffix])

      graph[srcReadID] = l


In [14]:
f = open('sub1.txt', 'w')

In [15]:
for x in graph:
  if(len(x)!=0):
    for y in x:
      f.write("S\t" + str(y[0]) + "\t*\t"+ "LN:i:" + str(y[1]) + '\n')

In [16]:
for i in range(len(graph)):
  if(len(graph[i]) != 0):
    for j in range(len(graph[i])):
      f.write("L\t" + str(i) + "\t+\t"+ str(graph[i][j][0]) + "\t+\t" + str(graph[i][j][1]) + "M" + '\n')